# Paquetes y Funciones

In [2]:
import nbimporter
from funciones_custom import *

# Avisos
import warnings
warnings.filterwarnings("ignore")

In [3]:
load_dotenv()

True

### API Keys

In [3]:
MAPS_API_KEY = os.getenv('MAPS_API_KEY')

# 1. Carga datos:

### Rutas

In [4]:
ruta_archivos_mapeo = os.getcwd().replace("Codigo\\Limpieza", "Datos\\mapping_files\\")
ruta_datos_sin_procesar = os.getcwd().replace("Codigo\\Limpieza", "Datos\\datos_sin_procesar\\")
ruta_datos_procesados = os.getcwd().replace("Codigo\\Limpieza", "Datos\\Procesados\\")

### Datos páginas de empleo

In [6]:
df_linkedin       = pd.read_csv(ruta_datos_sin_procesar + 'datos_linkedin.csv')
df_tecnoempleo    = pd.read_csv(ruta_datos_sin_procesar + 'datos_tecnoempleo.csv')
df_indeed         = pd.read_csv(ruta_datos_sin_procesar + 'datos_indeed.csv')
df_infojobs       = pd.read_csv(ruta_datos_sin_procesar + 'datos_infojobs.csv')
df_infoempleo     = pd.read_csv(ruta_datos_sin_procesar + 'datos_infoempleo.csv')
df_ticjob         = pd.read_csv(ruta_datos_sin_procesar + 'datos_ticjob.csv')
df_talenthacker   = pd.read_csv(ruta_datos_sin_procesar + 'datos_talenthacker.csv')

### Archivos de mapeo

In [33]:
with open(ruta_archivos_mapeo + "categorias_empleo.json", 'r') as archivo_json:
    mapping_titulo = json.load(archivo_json)

with open(ruta_archivos_mapeo + "jornadas.json", 'r') as archivo_json:
    mapping_jornada = json.load(archivo_json)

with open(ruta_archivos_mapeo + "contratos.json", 'r') as archivo_json:
    mapping_contrato = json.load(archivo_json)

with open(ruta_archivos_mapeo + 'ubicaciones.pickle', 'rb') as archivo:
    lista_ubicaciones = pickle.load(archivo)
    
with open(ruta_archivos_mapeo + 'herramientas.json', 'r') as archivo_json:
    dic_herramientas = json.load(archivo_json)
    
with open(ruta_archivos_mapeo + 'lista_herramientas.pickle', 'rb') as archivo:
    lista_herramientas = pickle.load(archivo)

with open(ruta_archivos_mapeo + 'empresas.json', 'r') as archivo_json:
    dic_empresas = json.load(archivo_json)

mapping_presencialidad  = {
    r'.*remoto.*'     : 'remoto',
    r'.*hibrido.*'    : 'hibrido',
    r'.*presencial.*' : 'presencial'}

columns = ["titulo"        , "empresa"      , "fecha",
           "herramientas"  , "descripcion"  , "ubicacion",
           "presencialidad", "funciones"    , "jornada",
           "experiencia"   , "tipo_contrato", "salario",
           "beneficios"    , "solicitudes"  , "fecha_scrapeo",
           "url"           , "portal"]

df_dic = {"linkedin"     : df_linkedin,   "tecnoempleo": df_tecnoempleo,
          "infojobs"     : df_infojobs,   "indeed"     : df_indeed,
          "infoempleo"   : df_infoempleo, "ticjob"     : df_ticjob,
          "talenthacker" : df_talenthacker}

columnas_numericas = ["experiencia", "solicitudes"]

lista_beneficios = ["beneficios", "pensiones", "pension", "cheque", "transporte", "transport", "cheques", "medical", "ticket", "tickets", "dental", "medico", "guarderia", "childcare", "child", "vida", "health"]

ciudades = ['cordoba', 'cartagena', 'santiago', 'linares', 'ibarra', "durango"]

pais = ', españa'

# 2. Clean

In [ ]:
##################################################################################### LIMPIEZA #################################################################################
preprocess_dataframes(df_dic, columns, columnas_numericas)

##################################################################################### MERGE DATOS ##############################################################################
merged_df = pd.concat(df_dic.values())
merged_df = merged_df[columns]
merged_df = merged_df.reset_index(drop=True)

##################################################################################### LIMPIEZA FECHAS ##########################################################################
merged_df['fecha_scrapeo'] = pd.to_datetime(merged_df['fecha_scrapeo'], format='%Y-%m-%d', errors='coerce').dt.date
merged_df['fecha'] = merged_df['fecha_scrapeo'] - merged_df['fecha'].apply(date_matching)
merged_df['fecha'] = pd.to_datetime(merged_df['fecha'], format='%Y-%m-%d', errors='coerce').dt.date

##################################################################################### LIMPIEZA TITULO ##########################################################################
merged_df['titulo'], merged_df['categoria_empleo'] = zip(*merged_df['titulo'].apply(lambda x: map_titulo(x, mapping_titulo)))

################################################################################# LIMPIEZA PRESENCIALIDAD ######################################################################
merged_df['presencialidad'] = merged_df['presencialidad'].apply(lambda x: map_presencialidad(x, mapping_presencialidad))
merged_df.loc[merged_df['ubicacion'].isin(['teletrabajo', '100 en remoto']), 'presencialidad'] = 'remoto'
merged_df["ubicacion"] = merged_df["ubicacion"].str.replace(r'\b(para|teletrabajo|100 en remoto)\b', '', regex=True).replace('', np.nan)

##################################################################################### LIMPIEZA JORNADA #########################################################################
merged_df['jornada'] = merged_df['jornada'].apply(lambda x: map_jornada(x, mapping_jornada))

##################################################################################### LIMPIEZA CONTRATO ########################################################################
merged_df['tipo_contrato'] = merged_df['tipo_contrato'].apply(lambda x: map_contrato(x, mapping_contrato))
merged_df = merged_df.replace('nan', np.nan)

##################################################################################### LIMPIEZA SALARIOS ########################################################################
merged_df['salario'] = merged_df['salario'].apply(lambda x: x if re.search(r"\d", str(x)) else np.nan)
rellenar_salarios(merged_df)

merged_df["salario"] = merged_df["salario"].str.replace(r'\b(y|a)\b', '-', regex=True)
merged_df["salario"] = merged_df["salario"].str.replace("k", '000')

merged_df['salario_min'], merged_df['salario_max'] = zip(*merged_df['salario'].apply(extraer_salarios))

##################################################################################### LIMPIEZA BENEFICIOS ######################################################################
merged_df["beneficios"] = merged_df.apply(lambda row: buscar_beneficios((str(row['beneficios']) + ', ' + str(row['descripcion'])), lista_beneficios), axis= 1)

merged_df = merged_df.map(convert_to_lowercase)

##################################################################################### LIMPIEZA HERRAMIENTAS ####################################################################
merged_df['herramientas'] = merged_df.apply(lambda row: [herramienta for herramienta in lista_herramientas if buscar_herramienta(str(row['herramientas']) + ', ' + str(row['descripcion']), herramienta)], axis=1)
merged_df['herramientas'] = merged_df['herramientas'].apply(lambda herramientas: set(key for herramienta in herramientas for key, values in dic_herramientas.items() if np.isin(herramienta, values, assume_unique = True)))

##################################################################################### LIMPIEZA EXPERIENCIA ####################################################################
merged_df['experiencia'] = merged_df.apply(extraer_experiencia, axis= 1)
merged_df = modificar_experiencia(merged_df)

##################################################################################### LIMPIEZA EMPRESAS #######################################################################
merged_df['empresa'] = merged_df['empresa'].apply(lambda x: next((key for key, values in dic_empresas.items() if x in values), x))

##################################################################################### LIMPIEZA UBICACIONES ####################################################################
asignar_ubicaciones(merged_df, lista_ubicaciones)

# Limpieza ubicacion con API
for ciudad in ciudades:
    merged_df.loc[merged_df['ubicacion'].str.contains(ciudad, na=False), 'ubicacion'] += pais

# Inicializamos API
gmaps = googlemaps.Client(key=MAPS_API_KEY)
df_final, ubicaciones_fallidas = obtener_ubicaciones(merged_df, gmaps, MAPS_API_KEY)

# Limpieza resultados API
df_final['comunidad'], df_final['pais'] = zip(*df_final.apply(agregar_galicia, axis=1))

df_final.loc[df_final['provincia'] == 'A Coruña', 'provincia'] = 'La Coruña'
df_final.loc[df_final['provincia'] == 'Seville', 'provincia'] = 'Sevilla'
df_final.loc[df_final['localidad'] == 'Seville', 'localidad'] = 'Sevilla'

df_final.loc[(df_final['latitud'].isnull()) & (df_final['comunidad'] == 'Galicia'), 'latitud'] = 42.755
df_final.loc[(df_final['longitud'].isnull()) & (df_final['comunidad'] == 'Galicia'), 'longitud'] = -7.866111

##################################################################################### DROPEAR COLUMNAS #########################################################################
df_final = df_final.drop(["funciones", "salario", "fecha_scrapeo", "ubicacion", "url"], axis = 1)

filtro_salario = (df_final['salario_min'] > 200_000) | (df_final['salario_max'] > 200_000)
df_final.loc[filtro_salario, ['salario_min', 'salario_max']] = np.nan
##################################################################################### SAVE #####################################################################################

df_final.to_csv(ruta_datos_procesados + 'datos_jobs_finales.csv', index=False)